### Resources

vegan subreddits:
- https://www.reddit.com/r/vegan/
- https://www.reddit.com/r/animalrights/
- https://www.reddit.com/r/animalwelfare/
- https://www.reddit.com/r/veg/
- https://www.reddit.com/r/vegetarian/
- https://www.reddit.com/r/vegetarianism/
- https://www.reddit.com/r/dietaryvegan/
- https://www.reddit.com/r/veganrecipes/
- https://www.reddit.com/r/vegproblems/

location-based vegan subreddits:
- https://www.reddit.com/r/VeganDenver/
- https://www.reddit.com/r/vegan/wiki/localvegansubreddits

### Next Steps

1. Get some Reddit data into a database
    - python scrape_reddit.py
    - eventually: have a script continuously running, scraping subreddit data as it comes in and updating with new comments and such
2. Create a pipeline for:
    - reading in the data
    - training a model on the data
    - saving the model out to disk
    - transforming the input data to topics and topic space
    - saving the transformed data out to disk